In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import ParameterGrid


ModuleNotFoundError: No module named 'sklearn'

Data Loading

In [ ]:
# === Load Data ===
df = pd.read_csv("traj.csv")  # or use your DataFrame directly

X = torch.tensor(df.iloc[:, :5].values, dtype=torch.float32)
y = torch.tensor(df.iloc[:, 5:].values, dtype=torch.float32)

dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=2)

Model Definition

In [ ]:
# === MLP Model ===
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

Training Process

In [ ]:
# === Training Function ===
def train_and_evaluate(hidden_dim, lr):
    model = MLP(input_dim=5, hidden_dim=hidden_dim, output_dim=2)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(20):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            val_loss += criterion(pred, yb).item()
    return val_loss

Parameter Search

In [ ]:
# === Hyperparameter Search ===
param_grid = {
    'hidden_dim': [8, 16, 32],
    'lr': [0.001, 0.01]
}

results = []
for params in ParameterGrid(param_grid):
    val_loss = train_and_evaluate(**params)
    print(f"Params: {params}, Val Loss: {val_loss:.4f}")
    results.append((params, val_loss))

# Best setting
best = sorted(results, key=lambda x: x[1])[0]
print("\nBest Params:", best[0], "with Validation Loss:", best[1])

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

# === Load Data ===
df = pd.read_csv("traj.csv")

X = torch.tensor(df.iloc[:, :5].values, dtype=torch.float32)
y = torch.tensor(df.iloc[:, 5:].values, dtype=torch.float32)

dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=2)

# === MLP Model ===
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# === Training Function with Loss Tracking ===
def train_and_evaluate(hidden_dim=16, lr=0.001, num_epochs=50):
    model = MLP(input_dim=5, hidden_dim=hidden_dim, output_dim=2)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        train_losses.append(epoch_train_loss / len(train_loader))

        # Validation
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                pred = model(xb)
                val_loss = criterion(pred, yb)
                epoch_val_loss += val_loss.item()
        val_losses.append(epoch_val_loss / len(val_loader))

    return train_losses, val_losses

# === Run Training ===
train_losses, val_losses = train_and_evaluate()

# === Plotting ===
plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("Training and Validation Loss (MLP)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/ipykernel/

KeyboardInterrupt: 